In [1]:
import cv2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw
from tqdm import tqdm
import os

file_info_df = pd.read_csv('../../dataset/EchoNet/FileInfo.csv')
tracings_df = pd.read_csv('../../dataset/EchoNet/VolumeTracings.csv')

def cycling_points(name, frame):
    tracings = tracings_df[tracings_df['FileName'] == name]    
    t = tracings[tracings['Frame'] == frame]
    points_l = list()
    points_r = list()
    for i in range(len(t)):
        c_1 = min(int(t.iloc[i]['X1'] + 0.5), 111)
        r_1 = min(int(t.iloc[i]['Y1'] + 0.5), 111)
        c_2 = min(int(t.iloc[i]['X2'] + 0.5), 111)
        r_2 = min(int(t.iloc[i]['Y2'] + 0.5), 111)
        if c_1 > c_2:
            c_1, c_2 = c_2, c_1
            r_1, r_2 = r_2, r_1
        points_l.append((c_1, r_1))
        points_r.append((c_2, r_2))
    points_l.sort(key=lambda x: x[1])
    points_r.sort(key=lambda x: x[1], reverse=True)
    points = points_l + points_r
    return points

In [2]:
# train data mask generation

train_df = file_info_df[file_info_df['Split'] == 'Train']

for name in tqdm(train_df['FileName']):
    esv_frame = train_df[train_df['FileName'] == name]['ESV'].values[0]
    edv_frame = train_df[train_df['FileName'] == name]['EDV'].values[0]
    img_path = f'../../dataset/EchoNet/Images/{name}'
    esv_img = cv2.imread(f'{img_path}/{esv_frame}.jpg', cv2.IMREAD_GRAYSCALE)
    edv_img = cv2.imread(f'{img_path}/{edv_frame}.jpg', cv2.IMREAD_GRAYSCALE)
    
    esv_poly_img = Image.new('RGB', esv_img.shape, (0,0,0))
    esv_draw = ImageDraw.Draw(esv_poly_img)
    esv_polygon_coords = cycling_points(f'{name}.avi', esv_frame)
    esv_draw.polygon(esv_polygon_coords, fill=(255, 255, 255))
    
    edv_poly_img = Image.new('RGB', edv_img.shape, (0,0,0))
    edv_draw = ImageDraw.Draw(edv_poly_img)
    edv_polygon_coords = cycling_points(f'{name}.avi', edv_frame)
    edv_draw.polygon(edv_polygon_coords, fill=(255, 255, 255))
    
    esv_poly_img = np.array(esv_poly_img)
    esv_poly_img[esv_poly_img < esv_poly_img.max()*0.9] = 0
    
    os.makedirs(f'../../dataset/EchoNet/Masks/{name}', exist_ok=True)
    cv2.imwrite(f'../../dataset/EchoNet/Masks/{name}/ESV_mask.jpg', np.array(esv_poly_img))
    cv2.imwrite(f'../../dataset/EchoNet/Masks/{name}/EDV_mask.jpg', np.array(edv_poly_img))

100%|██████████| 6018/6018 [09:54<00:00, 10.13it/s]


In [3]:
# Validation data mask generation
val_df = file_info_df[file_info_df['Split'] == 'Val']

for name in tqdm(val_df['FileName']):
    esv_frame = val_df[val_df['FileName'] == name]['ESV'].values[0]
    edv_frame = val_df[val_df['FileName'] == name]['EDV'].values[0]
    img_path = f'../../dataset/EchoNet/Images/{name}'
    esv_img = cv2.imread(f'{img_path}/{esv_frame}.jpg', cv2.IMREAD_GRAYSCALE)
    edv_img = cv2.imread(f'{img_path}/{edv_frame}.jpg', cv2.IMREAD_GRAYSCALE)
    
    esv_poly_img = Image.new('RGB', esv_img.shape, (0,0,0))
    esv_draw = ImageDraw.Draw(esv_poly_img)
    esv_polygon_coords = cycling_points(f'{name}.avi', esv_frame)
    esv_draw.polygon(esv_polygon_coords, fill=(255, 255, 255))
    
    edv_poly_img = Image.new('RGB', edv_img.shape, (0,0,0))
    edv_draw = ImageDraw.Draw(edv_poly_img)
    edv_polygon_coords = cycling_points(f'{name}.avi', edv_frame)
    edv_draw.polygon(edv_polygon_coords, fill=(255, 255, 255))
    
    esv_poly_img = np.array(esv_poly_img)
    esv_poly_img[esv_poly_img < esv_poly_img.max()*0.9] = 0
    
    os.makedirs(f'../../dataset/EchoNet/Masks/{name}', exist_ok=True)
    cv2.imwrite(f'../../dataset/EchoNet/Masks/{name}/ESV_mask.jpg', np.array(esv_poly_img))
    cv2.imwrite(f'../../dataset/EchoNet/Masks/{name}/EDV_mask.jpg', np.array(edv_poly_img))

  0%|          | 0/2006 [00:00<?, ?it/s]

100%|██████████| 2006/2006 [03:12<00:00, 10.42it/s]
